In [1]:
from qiskit import *
IBMQ.load_account()


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [2]:
provider = IBMQ.get_provider(group='open')
backend = provider.get_backend('ibmq_16_melbourne')
print(backend)

ibmq_16_melbourne


In [3]:
coupling_map=backend.configuration().coupling_map
num_qubits = backend.configuration().num_qubits
# print(coupling_map)
# print(num_qubits)
print(backend.configuration().gates)
# print(backend.configuration().local)
print(backend.configuration().max_shots)



[GateConfig(coupling_map=[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14]], name='id', parameters=[], qasm_def='gate id q { U(0,0,0) q; }'), GateConfig(coupling_map=[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14]], name='u1', parameters=['lambda'], qasm_def='gate u1(lambda) q { U(0,0,lambda) q; }'), GateConfig(coupling_map=[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14]], name='u2', parameters=['phi', 'lambda'], qasm_def='gate u2(phi,lambda) q { U(pi/2,phi,lambda) q; }'), GateConfig(coupling_map=[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14]], name='u3', parameters=['theta', 'phi', 'lambda'], qasm_def='gate u3(theta,phi,lambda) q { U(theta,phi,lambda) q; }'), GateConfig(coupling_map=[[0, 1], [0, 14], [1, 0], [1, 2], [1, 13], [2, 1], [2, 3], [2, 12], [3, 2], [3, 4], [3, 11], [4, 3], [4, 5], [4, 10], [5, 4], [5, 6], [5, 9], [6, 5], [6, 8], [7, 8], [8, 6], [8, 7]

In [4]:
prop= backend.properties()
print(prop.gate_property('u3',1))
print(prop.gate_property('u1',0))
print(prop.gate_property('u2',0))

{'gate_error': (0.0020615962165784207, datetime.datetime(2020, 5, 7, 7, 9, 21, tzinfo=datetime.timezone.utc)), 'gate_length': (1.0666666666666667e-07, datetime.datetime(2020, 5, 7, 10, 20, 21, tzinfo=datetime.timezone.utc))}
{'gate_error': (0.0, datetime.datetime(2020, 5, 7, 7, 8, 17, tzinfo=datetime.timezone.utc)), 'gate_length': (0.0, datetime.datetime(2020, 5, 7, 10, 20, 21, tzinfo=datetime.timezone.utc))}
{'gate_error': (0.0004937193995504997, datetime.datetime(2020, 5, 7, 7, 8, 17, tzinfo=datetime.timezone.utc)), 'gate_length': (5.3333333333333334e-08, datetime.datetime(2020, 5, 7, 10, 20, 21, tzinfo=datetime.timezone.utc))}


In [5]:
prop.readout_error(0)

0.02200000000000002

In [6]:

# backend_calib=Q_program.get_backend_calibration(backend=backend)
# print(backend_calib)
qubit_prop_map={}


for q in range(num_qubits):

    
    single_gate_error=max(prop.gate_error('u2',q),prop.gate_error('u3',q))
    readout_error=prop.readout_error(q)
    qubit_prop_map[q]=[(single_gate_error)]
    qubit_prop_map[q].append(readout_error)
    qubit_prop_map[q].append(prop.t1(q))
    qubit_prop_map[q].append(prop.t2(q))

print(qubit_prop_map)


qubit_adj_list=[]
n_edges = 0
for g in coupling_map:
    q1,q2=g
    print(q1,q2)
    qubit_adj_list.append((q1,q2,prop.gate_error('cx', [q1,q2])))
    n_edges +=1



n_qubits= num_qubits

print("printing info:")
print('qubits: ',num_qubits)
print('edges : ',n_edges)
print(qubit_prop_map)
print(qubit_adj_list)

# conf=open('config_'+str(backend_calib['last_update_date'][:10])+'_.txt','w')
conf=open('config_m.txt','w')
conf.write(str(n_qubits)+'\n')
conf.write(str(n_edges)+'\n')
for k in qubit_prop_map.keys():
    line=str(k)+" "+str(qubit_prop_map[k][0])+" "+str(qubit_prop_map[k][1])+" "+\
         str(qubit_prop_map[k][2])+" "+str(qubit_prop_map[k][3])+'\n'
    conf.write(line)

for mq in qubit_adj_list:
    line=str(mq[0])+" "+str(mq[1])+" "+str(mq[2])+'\n'
    conf.write(line)

conf.close()

{0: [0.000987195040255462, 0.02200000000000002, 7.724907597312889e-05, 1.9764837477388232e-05], 1: [0.0020615962165784207, 0.0675, 4.060782684653006e-05, 9.642094522494786e-05], 2: [0.007472444160717462, 0.02950000000000008, 4.493190887740298e-05, 9.525091346010214e-05], 3: [0.0007666415091074352, 0.05249999999999999, 7.864972664344837e-05, 5.591363519661963e-05], 4: [0.0037706991624312236, 0.019500000000000073, 5.855484653145938e-05, 2.8100909500855012e-05], 5: [0.005648014624650033, 0.06000000000000005, 1.889027712723862e-05, 3.326681115506176e-05], 6: [0.0034845005029349174, 0.03249999999999997, 7.777587658981836e-05, 8.208297220375773e-05], 7: [0.00264938534495629, 0.03049999999999997, 3.967660518814423e-05, 7.45172053593821e-05], 8: [0.0018602875167497146, 0.2889999999999999, 3.5691299612243416e-05, 6.179083484675874e-05], 9: [0.0027969462254865407, 0.03600000000000003, 3.6389214530520376e-05, 6.310287134125579e-05], 10: [0.004700129885763182, 0.031000000000000028, 4.5085913790199